In [ ]:
import darklight
import tangos

Import the simulation you'd like to run DarkLight on.  The following assumes that the database `Halo600.db` is in your current working directory.

If analzying EDGE halos, see `tutorial-edge.ipynb` for notes on additional support in DarkLight for retrieving EDGE data on DiRAC or the Surrey servers. 

In [ ]:
tangos.core.init_db('Halo600.db')
sim = tangos.get_simulation('Halo600_DMO')

This simulation contains lots of dark matter halos, but one giant 'main' halo.  The rest of the halos are much smaller. The simulations are run for the age of the universe, ~13.8 Gyr, and saves a bunch of 'snapshots' of the halo as it evolves over time. 

What we do here is get the last snapshot (timesteps[-1]), which corresponds to the present day (i.e. a redshift z = 0), and get the most massive halo (halos[0]), which will always be the first halo in the list of all halos, which is ordered by mass.

In [ ]:
halo = sim.timesteps[-1].halos[0]

Now that we have the halo, we can run DarkLight on it!

In [ ]:
t,z,vsmooth,sfh_insitu,mstar_insitu,mstar_total = darklight.DarkLight(halo)

The outputs are:

 - `t` = times [Gyr]
 - `z` = redshifts [unitless]
 - `vsmooth` = vmax over time, smoothed from simulation and suppressed to mimic lower vmaxes in hydro sims [km/s]
 - `sfh_insitu` = the star formation history of the galaxy that forms in this halo [Msun/yr]
 - `mstar_insitu` = integral of the (cumulative) in-situ star formation history [Msun]
 - `mstar_total` = total mass formed insitu + was accreted as little halos merged with this halo

All of these output arrays are of the same length.

Let's take a look at the star formation history and M* generated by DarkLight for this halo, and compare it against results from the full hydrodynamical simulations.  To do this comparison, we first have to get the halo from the corresponding hydro sim first:

In [ ]:
sim = darklight.edge.load_tangos_data('Halo600_fiducial')
hydro_halo = sim.timesteps[-1].halos[0]

I've created a built-in function that plots DarkLight results against those from the full hydro sims.  For EDGE, the stellar masses calculated by DarkLight usually match the values from hydro simulations to within a factor of two or so, but won't be an exact match.

In [ ]:
darklight.edge.plot_darklight_vs_edge_mstar(hydro_halo,t,z,vsmooth,sfh_insitu,mstar_total,mstar_insitu=mstar_insitu,figfn='halo'+shortname+'.pdf')

You can ask DarkLight to produce multiple realizations.  Generally, you'll find that if averaging multiple realizations will better match the EDGE simulations than if you created a single realization.  Here's an example running DarkLight with multiple realizations:

In [ ]:
t,z,vsmooth,sfh_insitu,mstar_insitu,mstar_total = darklight.DarkLight(halo, n=100)
darklight.edge.plot_darklight_vs_edge_mstar(hydro_halo,t,z,vsmooth,sfh_insitu,mstar_total,mstar_insitu=mstar_insitu,figfn='halo'+shortname+'-scatter.pdf')

In the above examples, I've run it with the default DarkLight settings, but you can play with these settings by supplying some keyword arguments. The arguments that modify the input parameters that control DarkLight's behavior and their default values are:
  
 - `vthres = 'falling'`, can also be a float [km/s]; the minimum vmax (the maximum velocity in the rotation curve) the halo must have to start forming stars again after reionization, given by a fit to the EDGE sims in Kim et al. 2024.
 - `zq = 4`, the redshift at which reionization quenching shuts star formation off
 - `occupation = 2.5e7` [msun] the occupation fraction (i.e. what halos have galaxies) to assume.  By default, assumes all halos with a mass above the given value (in solar masses) have galaxies.  The value used here reflects the mass down to which EDGE is resolved.  However, you can also ask DarkLight to assume all halos have galaxies (`'all'`), or use the occupation function from the EDGE1 sims (`'edge1'`), the EDGE1 RT sims (`'edge1rt'`), or a fit to observed Milky Way dwarfs (`'nadler20'`).
 - `prequench = 'fiducial'`, the SFR-vmax relation to use before reionization quenching
 - `postquench = 'schechter'`, the SFR-vmax relation to use after reionization quenching
 - `postquench_scatter='increasing'`, what scatter to assume for the SFR-vmax relation after reionization

There's also a few other parameters that you can modify:

 - `mapping = 3bins`, controls how to map the SFR-vmax relations into SFHs.  `3bins` assigns a single SFR before reionization quenching based on the average vmax in that period (bin 1), zero after reionization when the halo's vmax < vthres (bin 2), and a single SFR if vmax > vthres based on its vmax at z=0 (bin 3).  You can also specify `'all'` to have it update the SFR based on the vmax at each timestep. 
 - `timesteps = sim` [Gyr], the time resolution to produce the SFH and thus M*(t), which by default will use the timesteps in the simulation.  You can instead also specify a timestep in units of Gyr.
 - `mergers = True`, whether or not to include the contribution to M* from in-situ star formation, mergers, or both.  `True` includes both, `False` only the in-situ stars, and `'only'` only the accreted stars